In [1]:
import kagglehub
import pandas as pd
from pathlib import Path
import numpy as np

In [2]:
retail_path =  kagglehub.dataset_download("vijayuv/onlineretail")

100%|████████████████████████████████████| 7.20M/7.20M [00:00<00:00, 9.39MB/s]

Extracting files...


In [15]:
retail_csv = Path(retail_path).resolve() / 'OnlineRetail.csv'
retail_csv

PosixPath('/home/victor_he/.cache/kagglehub/datasets/vijayuv/onlineretail/versions/1/OnlineRetail.csv')

In [16]:
retail_ds = pd.read_csv(str(retail_csv), encoding='latin1')

# 1. Eploracion de Datos

## Verificar valores nulos

Existen 541 909 filas y 8 columnas, de las cuales hay muchas que contienen valores nulos

In [17]:
retail_ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [18]:
retail_ds.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


###  Se eliminan las filas que contiene valores nulos
Quedando solo 406 829 filas sin valores nulos

In [19]:
retail_ds.dropna(inplace=True)

In [20]:
retail_ds.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    406829 non-null  object 
 1   StockCode    406829 non-null  object 
 2   Description  406829 non-null  object 
 3   Quantity     406829 non-null  int64  
 4   InvoiceDate  406829 non-null  object 
 5   UnitPrice    406829 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      406829 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.9+ MB


## Eliminar facturas que comienzan con "C"
De la columna Description se elimina los valores que comiencen con c o C

In [21]:
retail_ds = retail_ds[ ~ retail_ds.InvoiceNo.str.startswith(('c', 'C'))]

Resultando en solo 397924 filas de facturas que no son cancelaciones

In [23]:
retail_ds.info()

<class 'pandas.core.frame.DataFrame'>
Index: 397924 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    397924 non-null  object 
 1   StockCode    397924 non-null  object 
 2   Description  397924 non-null  object 
 3   Quantity     397924 non-null  int64  
 4   InvoiceDate  397924 non-null  object 
 5   UnitPrice    397924 non-null  float64
 6   CustomerID   397924 non-null  float64
 7   Country      397924 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.3+ MB


## Crear una nueva columna llamada Total que sea Cantidad × Precio
Crear una nueva Columna 'Total' que es el resultado de multiplicar las columnas Quantity X UnitPrice

In [24]:
retail_ds['Total'] = retail_ds.Quantity * retail_ds.UnitPrice
retail_ds.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


# 2. Predicción: ¿El cliente hará una compra grande?

Crear columna 'Compra_Grande' , asignar 1 si el Total es mayor a 500, de lo contrario 0

In [25]:
retail_ds['Compra_Grande'] = [1 if t > 500 else 0 for t in retail_ds.Total]

In [26]:
retail_ds['Compra_Grande'] = retail_ds.Compra_Grande.astype(np.int8)

In [27]:
retail_ds.Compra_Grande.value_counts()

Compra_Grande
0    397011
1       913
Name: count, dtype: int64

In [28]:
retail_ds.info()

<class 'pandas.core.frame.DataFrame'>
Index: 397924 entries, 0 to 541908
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   InvoiceNo      397924 non-null  object 
 1   StockCode      397924 non-null  object 
 2   Description    397924 non-null  object 
 3   Quantity       397924 non-null  int64  
 4   InvoiceDate    397924 non-null  object 
 5   UnitPrice      397924 non-null  float64
 6   CustomerID     397924 non-null  float64
 7   Country        397924 non-null  object 
 8   Total          397924 non-null  float64
 9   Compra_Grande  397924 non-null  int8   
dtypes: float64(3), int64(1), int8(1), object(5)
memory usage: 30.7+ MB


## 2.1 Preprocesamiento
Extraer solo las columnas que tienen un significado o relacion con si es una compra grande o no.
Se seleccionan relacionadas al nombre del producto, cantidad, precio unitario y pais


In [29]:
datos_interes = retail_ds.loc[:, ['Description',	'Quantity',	'InvoiceDate',	'UnitPrice',	'CustomerID',	'Country','Compra_Grande']]
datos_interes.info()
datos_interes.InvoiceDate =  pd.to_datetime(datos_interes.InvoiceDate)
datos_interes.info()
datos_interes

<class 'pandas.core.frame.DataFrame'>
Index: 397924 entries, 0 to 541908
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Description    397924 non-null  object 
 1   Quantity       397924 non-null  int64  
 2   InvoiceDate    397924 non-null  object 
 3   UnitPrice      397924 non-null  float64
 4   CustomerID     397924 non-null  float64
 5   Country        397924 non-null  object 
 6   Compra_Grande  397924 non-null  int8   
dtypes: float64(2), int64(1), int8(1), object(3)
memory usage: 21.6+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 397924 entries, 0 to 541908
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Description    397924 non-null  object        
 1   Quantity       397924 non-null  int64         
 2   InvoiceDate    397924 non-null  datetime64[ns]
 3   UnitPrice      397924 non-null  float64       
 

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Compra_Grande
0,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,0
1,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0
2,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,0
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0
4,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0
...,...,...,...,...,...,...,...
541904,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,0
541905,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,0
541906,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,0
541907,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,0


### Seleccionar variables de Interés
**Del datetime es importante saber: que días de la semana en qué mes y a qué horas**
Estos son tomados como variables individuales que pueden afectar si una compra es grande o no

In [31]:
datos_interes['dia_semana'] = datos_interes.InvoiceDate.dt.weekday
datos_interes['mes'] = datos_interes.InvoiceDate.dt.month
datos_interes['hora'] = datos_interes.InvoiceDate.dt.hour
datos_interes = datos_interes[['Description',	'Quantity',	'UnitPrice',	'Country',	'dia_semana',	'mes','hora','Compra_Grande']]

**Otros variables son Descrption y Country** pero ya que los modelos escogidos solo aceptan variables numericas seria ideal transformar estas variables categoricas a variables numericas

In [32]:
datos_interes

,Description,Quantity,UnitPrice,Country,dia_semana,mes,hora,Compra_Grande
0,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,United Kingdom,2,12,8,0
1,WHITE METAL LANTERN,6,3.39,United Kingdom,2,12,8,0
2,CREAM CUPID HEARTS COAT HANGER,8,2.75,United Kingdom,2,12,8,0
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,United Kingdom,2,12,8,0
4,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,United Kingdom,2,12,8,0
...,...,...,...,...,...,...,...,...
541904,PACK OF 20 SPACEBOY NAPKINS,12,0.85,France,4,12,12,0
541905,CHILDREN'S APRON DOLLY GIRL,6,2.10,France,4,12,12,0
541906,CHILDRENS CUTLERY DOLLY GIRL,4,4.15,France,4,12,12,0
541907,CHILDRENS CUTLERY CIRCUS PARADE,4,4.15,France,4,12,12,0
